In [1]:
from import_data_encoded_knn import  *
from sequence_distance_matrix import *
from time import ctime


X_try, X_train, X_test, y_train, y_test, len_X_train = import_data_encoded(encoding = 'binary')

categorical = ['ring_number', "exposure_type", "conc1_type","species",'tripleBond', 'obs_duration_mean', 'doubleBond',
    'alone_atom_number', 'class', 'tax_order', 'family', 'genus', 'oh_count']

non_categorical =[ 'atom_number', 'bonds_number', 'Mol', 'MorganDensity', 'LogP']

chem = X_try.smiles

In [2]:
print(ctime())
dist_matr = basic_matrix(X_try, categorical, non_categorical, a_ham = 0.0015789473684210526)
print(ctime())

Fri Aug 14 11:26:02 2020
Fri Aug 14 11:33:46 2020


In [3]:
a_pub = 0.016206896551724137
print(ctime())
dist_matr += a_pub * pubchem2d_matrix(X_try)
print(ctime())

Fri Aug 14 11:34:48 2020
Fri Aug 14 11:55:19 2020


In [9]:
dist_matr = pd.DataFrame(dist_matr)

dist_matr_train = dist_matr.iloc[:len_X_train,:len_X_train]
dist_matr_test = dist_matr.iloc[len_X_train:,:len_X_train]

del dist_matr

In [12]:
neigh = KNeighborsClassifier(metric = 'precomputed', n_neighbors=1, n_jobs=-2, leaf_size=80)
neigh.fit(dist_matr_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=80, metric='precomputed',
                     metric_params=None, n_jobs=-2, n_neighbors=1, p=2,
                     weights='uniform')

In [13]:
from sklearn.metrics import accuracy_score
y_pred = neigh.predict(dist_matr_test)
accuracy_score(y_test, y_pred)

0.9055476159542679

In [16]:
# indici dei k composti più vicini ( k=1 )
neigh.kneighbors(dist_matr_test, return_distance = False)

array([[ 2210],
       [11094],
       [  968],
       ...,
       [   35],
       [12144],
       [ 5719]], dtype=int64)

In [180]:
X_test.index[0]

210

In [179]:
X_train.iloc[2210].name

214

In [188]:
uns_step = pd.DataFrame(neigh.kneighbors(dist_matr_test, return_distance = False), columns = ['idx_neigh_train'])

In [190]:
# uns_step = pd.DataFrame(uns_step.idx_neigh_train.apply(lambda x: X_train.iloc[x].name).values,
#                         columns = ['idx_train_neigh'])
uns_step

,idx_neigh_train
0,2210
1,11094
2,968
3,349
4,1158
...,...
7692,12683
7693,1021
7694,35
7695,12144


Devo prendere l'indice dell'esperimento test, l'indice e il label dell'esperimento train 

In [192]:
# uns_step = pd.concat([pd.DataFrame(X_test.index, columns = ['idx_test']), uns_step.idx_train_neigh], axis = 1)
uns_step = pd.concat([pd.DataFrame(X_test.index, columns = ['idx_test']), uns_step.idx_neigh_train], axis = 1)
uns_step

In [194]:
uns_step = pd.concat([uns_step, pd.DataFrame(y_train[uns_step.idx_neigh_train], columns = ['label_train'])], axis = 1)
uns_step

,idx_test,idx_neigh_train,label_train
0,210,2210,1
1,6586,11094,1
2,4516,968,1
3,13815,349,1
4,4556,1158,1
...,...,...,...
7692,12131,12683,0
7693,15545,1021,0
7694,17049,35,0
7695,11551,12144,0


In [195]:
uns_step.to_csv('dataset_prova/unsup_step_RASAR.csv')

# K = 3
### Matrice da rifare... però faccio il codice

In [168]:
neigh3 = KNeighborsClassifier(metric = 'precomputed', n_neighbors=3, n_jobs=-2, leaf_size=80)
neigh3.fit(dist_matr_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=80, metric='precomputed',
                     metric_params=None, n_jobs=-2, n_neighbors=3, p=2,
                     weights='uniform')

In [196]:
tmp = pd.DataFrame(neigh3.kneighbors(dist_matr_test, return_distance = False), columns = ['neigh1', 'neigh2', 'neigh3'])
tmp

,neigh1,neigh2,neigh3
0,2210,7761,7782
1,11094,14998,15437
2,968,4279,5919
3,349,10888,4271
4,1158,2371,15317
...,...,...,...
7692,12683,7189,15064
7693,1021,3672,13659
7694,35,6350,11594
7695,12144,656,1183


In [200]:
labels = pd.DataFrame({'label_neigh1': y_train[tmp.neigh1], 'label_neigh2': y_train[tmp.neigh1],
                       'label_neigh3': y_train[tmp.neigh3]})
labels

,label_neigh1,label_neigh2,label_neigh3
0,1,1,1
1,1,1,1
2,1,1,1
3,1,1,1
4,1,1,1
...,...,...,...
7692,0,0,0
7693,0,0,0
7694,0,0,1
7695,0,0,0


In [223]:
uns_step_k = labels.T.apply(pd.value_counts).T
uns_step_k

,0,1
0,NaN,3.0
1,NaN,3.0
2,NaN,3.0
3,NaN,3.0
4,NaN,3.0
...,...,...
7692,3.0,NaN
7693,3.0,NaN
7694,2.0,1.0
7695,3.0,NaN


In [224]:
uns_step_k = uns_step_k.fillna(0)
uns_step_k

,0,1
0,0.0,3.0
1,0.0,3.0
2,0.0,3.0
3,0.0,3.0
4,0.0,3.0
...,...,...
7692,3.0,0.0
7693,3.0,0.0
7694,2.0,1.0
7695,3.0,0.0


In [228]:
uns_step_k.columns = ['label_0_count', 'label_1_count']
uns_step_k

,label_0_count,label_1_count
0,0.0,3.0
1,0.0,3.0
2,0.0,3.0
3,0.0,3.0
4,0.0,3.0
...,...,...
7692,3.0,0.0
7693,3.0,0.0
7694,2.0,1.0
7695,3.0,0.0


In [229]:
uns_step_k.to_csv('dataset_prova/unsup_step_RASAR_k.csv')